In [1]:
from map import Map
from algoviz import AlgoViz
from algoviz.svg import SVGView, Circle, Rect, Text
from algoviz.gui import GUI, Button, Label, TextInput
import random, math, time
from random import randrange
from player import Player
from enemy import Enemy, Dummy
from item import Item


running = True
currentroom = 0
gegner = []
rooms = []
items = []

l = 1000  # spielfeldgröße
view = SVGView(l, l)


# hauptmethode zur verwaltung des spiels
def start_game():
    
     
    global currentroom, running, gegner, rooms, l, view, items
    
    feld = Rect(0, 0, l, l, view)
    feld.set_fill("white")
    
    # ertsellen der räume für diesen Spieldurchlauf
    rooms = create_game()
    
    pl = Player(l // 2, l // 2, 15, view)
    
    
    
    # zeichnen des ersten raums
    move_room()
    tutorial()
    pl.to_front()

    
    
    while running == True:
        jetzt = time.time()

        # spielerverwaltung - bewegung und überprüfen eines angriffes
        pl.change_speed()
        pl.check_wall_collision(rooms[currentroom], l)
        pl.bewegen()
        pl.attack(jetzt, gegner)

        # gegnerverwaltung:
        for i in range(len(gegner) -1, -1, -1):
            # lebt der gegner noch?
            if gegner[i].get_hp() <= 0:
                gegner[i]._kreis.set_fill("white")

                # war dies der letzte gegner wird das transport-Item generiert
                if len(gegner) == 1:
                    tile = update_tile(rooms, currentroom, l)
                    items.append(Item(gegner[i].get_start_x() - tile // 4, gegner[i].get_start_y() - tile // 4, view, "goal", "transport.png", (tile // 2), (tile // 2)))
                gegner[i].remove_healthbar()
                gegner.pop(i)
            
            else:
                # gegnerbewegung und schaden am spieler
                gegner[i].move(pl.get_x(), pl.get_y(), jetzt)
                if gegner[i].hits_player(pl.get_x(), pl.get_y()):
                    pl.is_hit(jetzt, 50)
                    if pl.get_hp() <= 0:
                        pl.update_healthbar()
                        game_over()
                        running = False


        # itemverwaltung
        for i in items:
            if i.is_collected(pl, i.get_x(), i.get_y(), i.get_width(), i.get_height(), view):
                # geschwindigkeits item
                if i.get_type() == "speed" and pl.get_speed() < 10:
                    pl.set_speed(pl.get_speed() + 1)

                # Lepenspunkte item
                elif i.get_type() == "hp" and pl.get_hp() <= 50:
                    pl.set_hp( pl.get_hp() + 50)
                    
                # Transport item, bringt dich in den nächsten Raum
                elif i.get_type() == "goal":
                    currentroom += 1
                    move_room()
                    end_text()
                    pl.move_to(l // 2, l // 2)
                    
                pl.to_front()
            
    

        # buttons für spielabbruch oder springen ins nächste level
        buttons(view.pressed_key(), pl)           
        view.sync()


In [2]:
def create_game():
    # erstellung des spiels - verschiedene Räume die nacheinander durchlaufen werden
    rooms = []

    # tutorial raum / lobby
    lobby = Map(8, 8)
    lobby.create_lobby()
    rooms.append(lobby)

    # random erstellte level
    for i in range(randrange(3, 5)):
        level = Map(15, 15)
        level.create_maze(random.randint(1, 3))
        rooms.append(level)

    # end raum
    end = Map(15, 15)
    end.create_end()
    rooms.append(end)

    return rooms

In [3]:
def buttons(key, pl):
    global running, currentroom
    # spielabbruch
    if key == "p":
        game_over()
        running = False

    # level überspringen
    if key == "l" and currentroom < len(rooms)-1:
        currentroom += 1
        move_room()
        end_text()
        
        pl.move_to(l // 2, l // 2)
        pl.to_front()

In [4]:
def move_room():
    # methode für den raumwechsel
    
    global currentroom, running, gegner, rooms, l, view, items
    rect = Rect(0, 0, l, l, view)
    rect.set_fill("white")

    # war dies der letzte raum -> spielende
    if currentroom >= len(rooms):
        running = False
        print("spiel vorbei")

    # zeichnen des nächsten raums
    else:
        
        rooms[currentroom].zeichnen(view, l)
        
        background = Rect(10, 20, 200, 50, view)
        background.set_fill("white")
        counter = Text(f"Raum: {currentroom} / {len(rooms)-1}",20,57,view, 35)

        # neue gegner und items erstellen, die alten löschen
        gegner = []
        items = []
        
        feld = rooms[currentroom]
        mazew = feld.get_length()
        mazeh = feld.get_height()
        tile = l // mazew
        
        for i in range(mazew):
             for j in range(mazeh):
                 # itemerstellung
                 if feld.layout[i][j] == 3: 
                     print("objekt")
                     if currentroom == 0:  
                         items.append(Item(j * (l // mazew)  + (tile // 4), i * (l // mazeh)  + (tile // 4), view, "speed", "speed.png", (tile // 2), (tile // 2)))
                     elif random.randint(1,2) == 1:
                         items.append(Item(j * (l // mazew)  + (tile // 4), i * (l // mazeh)  + (tile // 4), view, "speed", "speed.png", (tile // 2), (tile // 2)))
                     else:
                         items.append(Item(j * (l // mazew)  + (tile // 4), i * (l // mazeh)  + (tile // 4), view, "hp", "HP.png", (tile // 2), (tile // 2)))
                    

                 # gegnererstellung
                 if feld.layout[i][j] == 2: 
                     if currentroom == 0:
                         gegner.append(Dummy(j * (l // mazew) + (tile // 2), i * (l // mazeh) + (tile // 2), 15, view, feld))
                     else:
                         gegner.append(Enemy(j * (l // mazew) + (tile // 2), i * (l // mazeh) + (tile // 2), 15, view, feld, 50 * currentroom))
                     print("gegner erstellt")
                 
                

In [5]:
def update_tile(rooms, currentroom, l):
    # hilfsmethode um die pixelgröße der tiles zu berechnen
    mazew = rooms[currentroom].get_length()
    
    return l // mazew

In [6]:
def tutorial():
    tut1 = Text("1. Willkommen!", 330, 600, view)
    tut2 = Text("Das hier ist der Tutorialraum.", 290, 625, view)
    tut3 = Text("Bewege dich mit wasd und q, e, y und c", 260, 650, view)
    tut4 = Text("Lies erstmal alles durch bevor es richtig losgeht...", 225, 675, view)
    tut5 = Text("⟵ Sammel dieses Objekt ein, um permanent schneller zu werden.", 250, 190, view)
    tut6 = Text("Das ist ein Gegner. Dieser ist dumm und und kann nichts, berühre die echten später ´", 230, 785, view)
    tut7 = Text("aber nicht, sonst aua. Nutze Leerzeichen, um anzugreifen. Der rote Kreis, der dabei", 230, 810, view)
    tut8 = Text("⟵  erscheint, ist dein Attackenradius. Sammel das Item auf, das er droppt, um in den ersten", 203, 835, view)
    tut9 = Text("Raum zu gelangen. Schaffe es durch alle Räume, um zu gewinnen!", 230, 860, view)
    tut10 = Text("2.", 277, 165, view)
    tut11 = Text("3.", 230, 760, view)

In [7]:
def end_text():
    # spiel gewonnen:
    if len(rooms) - 1 == currentroom:
        text1 = Text("Herzlichen Glückwunsch! Du hast das Spiel gewonnen!", 100, 150, view, 30)
        text2 = Text("Als Belohnung gewinnst du die Antwort auf die Frage nach dem Leben,", 100, 200, view, 25)
        text3 = Text("dem Universum und dem ganzen Rest:", 100, 250, view, 25)

In [8]:
def game_over():
    rect = Rect(0, 0, l, l, view)
    rect.set_fill("white")
    text = Text("Game Over!", 250, 400, view, 90)
    

In [9]:
start_game()

objekt
gegner erstellt
gegner erstellt
objekt
gegner erstellt
objekt
gegner erstellt
gegner erstellt
gegner erstellt
objekt
gegner erstellt
gegner erstellt
objekt
gegner erstellt
objekt
gegner erstellt
objekt
